1. How many customers do we have in the data?

In [ ]:
SELECT COUNT (DISTINCT customer_id) AS number_of_customer
FROM customers
;

Answer: 795 customers

2. What was the city with the most profit for the company in 2015?

3. In 2015, what was the most profitable city's profit?

In [ ]:
SELECT
   SUM (order_profits) AS total_profit,
   shipping_city
FROM order_details
JOIN orders USING (order_id)
WHERE EXTRACT (YEAR FROM order_date)=2015
GROUP BY 2
ORDER BY total_profit DESC
;

Answer: New York City with total profit of 14753

4. How many different cities do we have in the data?

In [ ]:
SELECT COUNT (DISTINCT shipping_city)
FROM orders
;

Answer: 531 city

5. Show the total spent by customers from low to high.

In [ ]:
SELECT
      cu.customer_id,
      cu.customer_name,
      SUM (od.order_sales) AS spend_per_cust
FROM customers AS cu
JOIN orders AS ord USING (customer_id)
JOIN order_details AS od USING (order_id)
GROUP BY 1,2
ORDER BY 3
;

6.What is the most profitable city in the State of Tennessee?

In [ ]:
SELECT
   SUM (order_profits) AS total_profit,
   shipping_city
FROM order_details
JOIN orders USING (order_id)
WHERE shipping_state='Tennessee'
GROUP BY 2, shipping_state
ORDER BY total_profit DESC
;

Answer: Lebanon

7. What’s the average annual profit for that city across all years?

In [ ]:
WITH yearly_profit AS (
      SELECT
            SUM (order_profits) AS profit_sum,
            EXTRACT (YEAR FROM order_date)
      FROM order_details
      JOIN orders USING (order_id)
      WHERE shipping_city='Lebanon'
      GROUP BY 2
   )
SELECT AVG(profit_sum) AS avg_profit_per_year
FROM yearly_profit
;

Answer: 27.66/year

8. What is the distribution of customer types in the data?

In [ ]:
--in %
WITH total_cust AS (
    SELECT COUNT (customer_segment) AS total_count
    FROM customers
   )
SELECT
   ROUND(COUNT (customer_segment)*100.0/(SELECT total_count FROM total_cust),2) AS perc_per_segment,
      customer_segment
FROM customers
GROUP BY 2
;

--in numbers
SELECT
   COUNT (customer_segment) AS total_count,
   customer_segment
FROM customers
GROUP BY 2
;

9.What’s the most profitable product category on average in Iowa across all years?

In [ ]:
WITH profit_per_year AS (
      SELECT
            SUM (order_profits) total_profit,
            product_category,
            EXTRACT (YEAR FROM order_date) AS YEAR
      FROM order_details
      JOIN product USING (product_id)
      JOIN orders USING (order_id)
      WHERE shipping_state ='Iowa'
      GROUP BY 2, 3
      ORDER BY 1 DESC
      )
SELECT
   AVG (total_profit) AS avg_year_profit,
   product_category
FROM profit_per_year
GROUP BY 2
ORDER BY 1 DESC
;

Answer: Furniture

10. What is the most popular product in that category across all states in 2016?

In [ ]:
SELECT
   product_name,
   SUM (quantity) AS total_quantity
FROM product
FULL JOIN order_details USING (product_id)
FULL JOIN orders USING (order_id)
WHERE product_category ='Furniture' AND EXTRACT (YEAR FROM order_date)=2016
GROUP BY 1
ORDER BY total_quantity DESC
;

Answer: Global Push Button Manager's Chair, Indigo

11. Which customer got the most discount in the data? (in total amount)

In [ ]:
WITH full_price AS(
      SELECT
            order_sales/(1-order_discount) AS full_price,
       order_sales,
            customer_id,
            customer_name
      FROM order_details
      JOIN orders USING (order_id)
      JOIN customers USING (customer_id)
   )
SELECT
    SUM(full_price-order_sales) AS total_disc,
    customer_id,
      customer_name
FROM full_price
GROUP BY 2,3
ORDER BY 1 DESC
;

Answer: 687 Sean Miller

12. How widely did monthly profits vary in 2018?

In [ ]:
WITH month_profit AS (
      SELECT
            SUM (order_profits) AS month_profit,
            EXTRACT (MONTH FROM order_date) AS month_of_2018
      FROM order_details
      JOIN orders USING (order_id)
      GROUP BY order_date
      HAVING EXTRACT (YEAR FROM order_date)=2018
   ),
profit AS (
      SELECT
            SUM(month_profit) AS profit,
            month_of_2018
      FROM month_profit
      GROUP BY 2
    )
SELECT
   SUM(profit) AS profit,
   SUM(profit)-(LAG(SUM(profit)) OVER (ORDER BY month_of_2018)) AS difference,
      month_of_2018
FROM profit
GROUP BY 3
;

13. Which was the biggest order regarding sales in 2015?

In [ ]:
SELECT
   MAX (order_sales) AS max_sales,
   order_id
FROM order_details
JOIN orders USING (order_id)
GROUP BY 2, order_date
HAVING EXTRACT (YEAR FROM order_date)=2015
ORDER BY max_sales DESC
;
-- 14. What was the rank of each city in the East region in 2015 in quantity?
-- 1st New York City 1708
SELECT
   SUM (quantity) AS sum_quant,
   shipping_city,
      RANK() OVER (ORDER BY SUM(quantity) DESC) AS rank
FROM orders
JOIN order_details USING (order_id)
WHERE shipping_region ='East' AND EXTRACT (YEAR FROM order_date)=2015
GROUP BY 2
ORDER BY sum_quant DESC
;

Answer: CA-2015-145317, 22638

14. What was the rank of each city in the East region in 2015 in quantity?

In [ ]:
SELECT
   SUM (quantity) AS sum_quant,
   shipping_city,
      RANK() OVER (ORDER BY SUM(quantity) DESC) AS rank
FROM orders
JOIN order_details USING (order_id)
WHERE shipping_region ='East' AND EXTRACT (YEAR FROM order_date)=2015
GROUP BY 2
ORDER BY sum_quant DESC
;

Answer: 1st was New York City, 1708

15. Display customer names for customers who are in the segment ‘Consumer’ or ‘Corporate.’

How many customers are there in total?

In [ ]:
SELECT customer_name
FROM customers
WHERE customer_segment IN ('Consumer', 'Corporate')
;

Answer: 647

16. Calculate the difference between the largest and smallest order quantities for product id ‘100.’

In [ ]:
WITH max_min AS(
     SELECT
            MIN (quantity) AS min_quant,
            MAX (quantity) AS max_quant
      FROM order_details
      WHERE product_id=100
   )
SELECT max_quant-min_quant AS diff
FROM max_min
;

Answer: 4

17. Calculate the percent of products that are within the category ‘Furniture.’

In [ ]:
WITH total_category AS (
    SELECT COUNT (product_category) AS total_count_categ
    FROM product
   )
SELECT
   ROUND (COUNT (product_name)*100.0/(SELECT total_count_categ FROM total_category),2) AS perc_from_total
FROM product
WHERE product_category='Furniture'
;

Answer: 20.54%

18. Display the number of product manufacturers with more than 1 product in the product table.

In [ ]:
WITH manufacturer AS (
      SELECT
            COUNT (product_name) AS numb_of_prod,
            product_manufacturer
      FROM product
      GROUP BY 2
      HAVING COUNT (product_name)>1
    )
SELECT COUNT (product_manufacturer) AS total_manufacturers
FROM manufacturer
;

Answer: 169

19. Find what product manufacturers has more than 2 products.

Ex: A product with an identical product manufacturer can be considered a duplicate.

In [ ]:
      SELECT
            COUNT (product_name) AS numb_of_prod,
            product_manufacturer
      FROM product
      GROUP BY 2
      HAVING COUNT (product_name)>2
;

20. Show the product_subcategory and the total number of products in the subcategory.

Show the order from most to least products and then by product_subcategory name ascending.

In [ ]:
SELECT
   product_subcategory,
   COUNT (product_name) AS products_in_subcat
FROM product
GROUP BY 1
ORDER BY products_in_subcat DESC, product_subcategory
;

21. Show the product_id(s), the sum of quantities, where the total sum of it's product quantities is greater than or equal to 100.

In [ ]:
SELECT
   product_id,
   SUM (quantity) AS order_quant
FROM product
JOIN order_details USING (product_id)
GROUP BY product_id
HAVING SUM (quantity)>=100
ORDER BY order_quant DESC
;

22. Show the product_id(s), the sum of quantities, where for each sale of product quantities is greater than or equal to 100.

In [ ]:
SELECT
   product_id,
   SUM (quantity) AS order_quant
FROM product
JOIN order_details USING (product_id)
WHERE quantity>=100
GROUP BY product_id
ORDER BY order_quant DESC
;